# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

In [8]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

In this case, I used a 30 Day trial version.

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks. Also, I have ignored 'DISCA', 'HFC','VIAC','WLTW' from the tickers as they are discountinued from the S&P 500.

In [9]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. I used a 30 Days trial period as the SandBox is deprecated by IEX Cloud


In [10]:
from secret import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [11]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 74661316, 'calculationPrice': 'close', 'change': 2.84, 'changePercent': 0.01881, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 153.845, 'iexCloseTime': 1676321999897, 'iexLastUpdated': 1676321999897, 'iexMarketPercent': 52.938762143870754, 'iexOpen': 150.96, 'iexOpenTime': 1676298600141, 'iexRealtimePrice': 153.84, 'iexRealtimeSize': 100, 'iexVolume': 969944, 'lastTradeTime': 1676321999900, 'latestPrice': 153.85, 'latestSource': 'Close', 'latestTime': 'February 13, 2023', 'latestUpdate': 1676322000811, 'latestVolume': None, 'low': None, 'lowSource': None, 'lowTime': None, 'marketCap': 2434206392100, 'oddLotDelayedPrice': Non

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [12]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [13]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [14]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index = True

)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,153.85,2434206392100,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [15]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:10]:
    api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
    [
        stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ],
    index = my_columns
    ),
ignore_index = True
    )
    

In [16]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,154.60,45739967595,N/A
1,AAL,16.51,10729863644,N/A
2,AAP,152.19,9149394793,N/A
3,AAPL,153.85,2434206392100,N/A
4,ABBV,153.73,271868508495,N/A
5,ABC,158.61,32080171199,N/A
6,ABMD,381.02,17180642928,N/A
7,ABT,108.52,189212626280,N/A
8,ACN,288.35,189846327724,N/A
9,ADBE,375.00,171675000000,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [17]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [18]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
        ],
        index = my_columns
        ),
        ignore_index = True
        )
        
    print(final_dataframe)
    

   Ticker  Stock Price Market Capitalization Number of Shares to Buy
0       A       154.60           45739967595                     N/A
1     AAL        16.51           10729863644                     N/A
2     AAP       152.19            9149394793                     N/A
3    AAPL       153.85         2434206392100                     N/A
4    ABBV       153.73          271868508495                     N/A
..    ...          ...                   ...                     ...
95   CINF       127.11           19979697390                     N/A
96     CL        73.77           61613752272                     N/A
97    CLX       152.89           18885726242                     N/A
98    CMA        75.99            9951074320                     N/A
99  CMCSA        39.09          164436461243                     N/A

[100 rows x 4 columns]
    Ticker  Stock Price Market Capitalization Number of Shares to Buy
0        A       154.60           45739967595                     N/A
1      A

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [19]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,154.60,45739967595,N/A
1,AAL,16.51,10729863644,N/A
2,AAP,152.19,9149394793,N/A
3,AAPL,153.85,2434206392100,N/A
4,ABBV,153.73,271868508495,N/A
...,...,...,...,...
496,YUM,132.52,37329276002,N/A
497,ZBH,128.05,26871598796,N/A
498,ZBRA,318.95,16467324072,N/A
499,ZION,52.90,7914779187,N/A


In [20]:
portfolio_size = input('Enter the value of your portfolio: ')

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a Number! \nPlease Try again:")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

In [21]:
position_size = val/len(final_dataframe.index)
for i in range (0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,154.60,45739967595,1
1,AAL,16.51,10729863644,12
2,AAP,152.19,9149394793,1
3,AAPL,153.85,2434206392100,1
4,ABBV,153.73,271868508495,1
...,...,...,...,...
496,YUM,132.52,37329276002,1
497,ZBH,128.05,26871598796,1
498,ZBRA,318.95,16467324072,0
499,ZION,52.90,7914779187,3


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [22]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine ='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
{
    'font_color': font_color,
    'bg_color':background_color,
    'border': 1
}
)
dollar_format = writer.book.add_format(
{
    'num_format':'$0.00',
    'font_color': font_color,
    'bg_color':background_color,
    'border': 1
}
)
integer_format = writer.book.add_format(
{
    'num_format':'0',
    'font_color': font_color,
    'bg_color':background_color,
    'border': 1
}
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.



In [25]:
column_formats = {
    
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [26]:
writer.save()